<a href="https://colab.research.google.com/github/ErikaCibellySx24/analise-comercial-imobiliaria/blob/main/automacao_imobiliaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desenvolvimento de database e automação no envio de informações

## Súmario


1.   Importando bibliotecas
2.   Tratamento de dados
3.   Criando automação na exportação dos dados





In [5]:
pip install pandas faker openpyxl

  Using cached faker-37.4.0-py3-none-any.whl.metadata (15 kB)
Using cached faker-37.4.0-py3-none-any.whl (1.9 MB)


In [6]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [7]:
# Inicializações
fake = Faker('pt_BR')
random.seed(42)
np.random.seed(42)

# Parâmetros
n_dados = 3000
n_diretorias = 5
n_lideres_por_diretoria = 2
n_corretores_por_lider = 2

# Gerar nomes de diretorias, líderes e corretores
diretorias = [f'Diretoria {i+1}' for i in range(n_diretorias)]
lideres = []
corretores = []

for diretoria in diretorias:
    for _ in range(n_lideres_por_diretoria):
        lider = fake.name()
        lideres.append({'lider': lider, 'diretoria': diretoria})
        for _ in range(n_corretores_por_lider):
            corretor = fake.name()
            corretores.append({'corretor': corretor, 'lider': lider, 'diretoria': diretoria})

# Lista de canais e motivos de não conversão
canais = ['Instagram', 'Facebook', 'Google Ads', 'Indicação', 'Site', 'Whatsapp']
motivos_nao_conversao = [
    'Sem interesse no momento',
    'Preço acima do esperado',
    'Financiamento negado',
    'Não encontrou imóvel adequado',
    'Demora no atendimento',
    'Problemas na documentação'
]

# Gerar dados entre janeiro e maio de 2025
inicio = datetime(2025, 1, 1)
fim = datetime(2025, 5, 31)

# Gerar dados
dados = []

for _ in range(n_dados):
    corretor_info = random.choice(corretores)
    corretor = corretor_info['corretor']
    lider = corretor_info['lider']
    diretoria = corretor_info['diretoria']

    data_lead = fake.date_between(start_date=inicio, end_date=fim)
    lead_convertido = np.random.rand() < 0.6

    imovel_negociado = 1 if lead_convertido else 0
    comissao = round(np.random.uniform(3000, 15000), 2) if lead_convertido else 0

    if lead_convertido:
        percentual_comissao = np.random.uniform(0.03, 0.06)
        valor_venda = round(comissao / percentual_comissao, 2)
        tempo_fechamento = random.randint(5, 30)
        data_venda = data_lead + timedelta(days=tempo_fechamento)
    else:
        percentual_comissao = None
        valor_venda = 0
        tempo_fechamento = None
        data_venda = None

    canal = random.choice(canais)
    motivo_nao = random.choice(motivos_nao_conversao) if not lead_convertido else None

    dados.append({
        'diretoria': diretoria,
        'lider': lider,
        'corretor': corretor,
        'data_lead': data_lead,
        'imovel_negociado': imovel_negociado,
        'comissao': comissao,
        'percentual_comissao': round(percentual_comissao * 100, 2) if percentual_comissao else None,
        'valor_venda': valor_venda,
        'tempo_fechamento_dias': tempo_fechamento,
        'data_venda': data_venda,
        'canal_origem': canal,
        'motivo_nao_conversao': motivo_nao
    })

# Criar DataFrame
df = pd.DataFrame(dados)

# Calcular taxa de conversão por corretor
conversion_rate = df.groupby('corretor')['imovel_negociado'].mean().round(2).rename('taxa_conversao')
df = df.merge(conversion_rate, on='corretor')

# Exportar todos os dados
df.to_excel("vendas_imobiliaria_jan_mai_2025.xlsx", index=False)

# Filtrar somente vendas realizadas (imóvel negociado)
df_vendas = df[df['imovel_negociado'] == 1]

# Exportar somente as vendas confirmadas
df_vendas.to_excel("vendas_confirmadas_jan_mai_2025.xlsx", index=False)

# Mostrar as primeiras linhas para conferir
print(df_vendas[['data_lead', 'data_venda', 'tempo_fechamento_dias']].head(10))


     data_lead  data_venda  tempo_fechamento_dias
0   2025-01-18  2025-01-23                    5.0
1   2025-01-09  2025-01-21                   12.0
2   2025-05-25  2025-06-22                   28.0
4   2025-02-21  2025-02-27                    6.0
5   2025-02-07  2025-02-18                   11.0
6   2025-05-23  2025-06-16                   24.0
7   2025-03-13  2025-03-24                   11.0
8   2025-05-05  2025-05-23                   18.0
10  2025-04-10  2025-05-09                   29.0
11  2025-05-27  2025-06-11                   15.0


In [8]:
dados_vendas = pd.read_excel('vendas_confirmadas_jan_mai_2025.xlsx')

In [9]:
dados_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810 entries, 0 to 1809
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   diretoria              1810 non-null   object        
 1   lider                  1810 non-null   object        
 2   corretor               1810 non-null   object        
 3   data_lead              1810 non-null   datetime64[ns]
 4   imovel_negociado       1810 non-null   int64         
 5   comissao               1810 non-null   float64       
 6   percentual_comissao    1810 non-null   float64       
 7   valor_venda            1810 non-null   float64       
 8   tempo_fechamento_dias  1810 non-null   int64         
 9   data_venda             1810 non-null   datetime64[ns]
 10  canal_origem           1810 non-null   object        
 11  motivo_nao_conversao   0 non-null      float64       
 12  taxa_conversao         1810 non-null   float64       
dtypes: 

In [10]:
dados_vendas.describe()

,data_lead,imovel_negociado,comissao,percentual_comissao,valor_venda,tempo_fechamento_dias,data_venda,motivo_nao_conversao,taxa_conversao
count,1810,1810.0,1810.000000,1810.000000,1810.000000,1810.000000,1810,0.0,1810.000000
mean,2025-03-17 07:45:24.861878528,1.0,8904.942210,4.502773,205977.719680,17.599448,2025-04-03 22:08:37.127071744,NaN,0.606497
min,2025-01-01 00:00:00,1.0,3000.370000,3.000000,50797.360000,5.000000,2025-01-06 00:00:00,NaN,0.520000
25%,2025-02-08 00:00:00,1.0,5820.910000,3.750000,130464.642500,11.000000,2025-02-24 00:00:00,NaN,0.580000
50%,2025-03-19 00:00:00,1.0,9016.755000,4.480000,199780.640000,18.000000,2025-04-05 00:00:00,NaN,0.610000
75%,2025-04-25 00:00:00,1.0,11896.805000,5.277500,270563.065000,24.000000,2025-05-12 00:00:00,NaN,0.630000
max,2025-05-30 00:00:00,1.0,14986.710000,6.000000,467763.050000,30.000000,2025-06-28 00:00:00,NaN,0.680000
std,NaN,0.0,3497.944762,0.873987,92721.751528,7.498244,NaN,NaN,0.039309


In [11]:
dados_vendas.shape

(1810, 13)

In [12]:
dados_vendas.head(1-2)

,diretoria,lider,corretor,data_lead,imovel_negociado,comissao,percentual_comissao,valor_venda,tempo_fechamento_dias,data_venda,canal_origem,motivo_nao_conversao,taxa_conversao
0,Diretoria 1,Larissa Leão,Danilo Pereira,2025-01-18,1,14408.57,5.20,277302.16,5,2025-01-23,Whatsapp,NaN,0.59
1,Diretoria 3,Heloísa Duarte,Luiz Fernando Ferreira,2025-01-09,1,4872.22,3.47,140491.44,12,2025-01-21,Facebook,NaN,0.63
2,Diretoria 2,Sabrina Nascimento,Isadora da Conceição,2025-05-25,1,13394.11,4.80,278849.63,28,2025-06-22,Instagram,NaN,0.66
3,Diretoria 4,Dr. José Pedro Araújo,Sra. Heloisa Moraes,2025-02-21,1,14638.92,5.50,266291.55,6,2025-02-27,Instagram,NaN,0.54
4,Diretoria 1,Larissa Leão,Vitória Ramos,2025-02-07,1,5181.90,3.55,145960.23,11,2025-02-18,Facebook,NaN,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804,Diretoria 4,Dr. José Pedro Araújo,Leandro Camargo,2025-04-25,1,9401.76,4.79,196312.49,15,2025-05-10,Site,NaN,0.57
1805,Diretoria 1,Larissa Leão,Danilo Pereira,2025-03-05,1,12685.28,4.57,277585.82,30,2025-04-04,Indicação,NaN,0.59
1806,Diretoria 1,Larissa Leão,Danilo Pereira,2025-05-10,1,4548.78,4.61,98659.66,27,2025-06-06,Whatsapp,NaN,0.59
1807,Diretoria 5,Dr. Davi Luiz Sousa,Nina Garcia,2025-03-21,1,13434.49,5.29,253850.38,30,2025-04-20,Whatsapp,NaN,0.64


In [13]:
dados_vendas.isnull().sum()


,0
diretoria,0
lider,0
corretor,0
data_lead,0
imovel_negociado,0
comissao,0
percentual_comissao,0
valor_venda,0
tempo_fechamento_dias,0
data_venda,0
